In [4]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import io
import os
from google.cloud import storage

In [11]:
def extract_and_load_to_storage(folder_id):
    # Uso de las credenciales de autenticación para acceder al servidor de Google Cloud
    credentials = service_account.Credentials.from_service_account_file(
        '../Clave/proyecto-nuevo-423502-b27ede29c633.json',
        scopes=['https://www.googleapis.com/auth/drive']
    )

    # Se instancia el cliente de la API de Google Drive
    drive_service = build('drive', 'v3', credentials=credentials)

    # Se obtiene la lista de archivos desde la carpeta especificada en Google Drive
    query = f"'{folder_id}' in parents"
    response = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = response.get('files', [])

    # Verificar si hay más archivos disponibles en el resultado y obtenerlos si es necesario
    while 'nextPageToken' in response:
        # Realizar la solicitud para la siguiente lista de archivos utilizando el token de paginación
        response = drive_service.files().list(pageSize=1000, fields="files(id, name)", pageToken=response['nextPageToken']).execute()
        
        # Agregar los archivos de la siguiente página a la lista de archivos
        files.extend(response.get('files', []))

    # Configurar cliente de Google Cloud Storage
    storage_client = storage.Client()

    # Nombre del bucket en Google Cloud Storage
    bucket_name = "archivos-originales"

    # Iterar sobre los archivos obtenidos y cargarlos en Google Cloud Storage
    for file in files:
        file_id = file['id']
        file_name = file['name']

        # Descargar el archivo desde Google Drive
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = googleapiclient.http.MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()

        # Cargar el archivo descargado en Google Cloud Storage
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(file_name)
        blob.upload_from_file(fh)

        print(f"Archivo '{file_name}' cargado en Google Cloud Storage.")



In [12]:
# Llamar a la función principal
extract_and_load_to_storage("1TI-SsMnZsNP6t930olEEWbBQdo_yuIZF")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.